In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
#Loading dataset
data = pd.read_csv('fer2013.csv')
pixels = data['pixels'].tolist()

# Pre-processing: each pixels array is processed into a numpy array and added to the set
X = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(48, 48)
    face = np.expand_dims(face, axis=-1)
    X.append(face)

X = np.asarray(X)
X = X.astype('float32')
X /= 255.0  # Normlization [0,1]

# Getting the predicted labeles from the set
y = pd.get_dummies(data['emotion']).values  # One-hot encoding

# Divisione in train e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
#CNN model
#Sequential() --> stack of linear layer with one input and one output
model = tf.keras.models.Sequential([
    #Conv2D() conv levev with (num of filters, dim of filters, activate function, input shape --> 48X48 gray scale)
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    #MaxPooling2D() reducing the spacial dimension of the output (info preserved)
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # Flatten() the 2D matrix of pixel is flattened into a 1D vector
    tf.keras.layers.Flatten(),
    #Dense(n, act) level denselt connected with n neuron with activation function act
    tf.keras.layers.Dense(256, activation='relu'),
    #Dropout(perc) used to prevent overfitting: it turn off casually perc neurons
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\acaia\anaconda3\envs\first_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
#Fitting the model
history = model.fit(X_train, y_train, epochs=60, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/60
449/449 ━━━━━━━━━━━━━━━━━━━━ 49s 107ms/step - accuracy: 0.2516 - loss: 1.8019 - val_accuracy: 0.3975 - val_loss: 1.5373
Epoch 2/60
449/449 ━━━━━━━━━━━━━━━━━━━━ 47s 104ms/step - accuracy: 0.4151 - loss: 1.5107 - val_accuracy: 0.4709 - val_loss: 1.3736
Epoch 3/60
449/449 ━━━━━━━━━━━━━━━━━━━━ 47s 104ms/step - accuracy: 0.4830 - loss: 1.3541 - val_accuracy: 0.5017 - val_loss: 1.3167
Epoch 4/60
449/449 ━━━━━━━━━━━━━━━━━━━━ 47s 105ms/step - accuracy: 0.5177 - loss: 1.2666 - val_accuracy: 0.5391 - val_loss: 1.2219
Epoch 5/60
449/449 ━━━━━━━━━━━━━━━━━━━━ 48s 106ms/step - accuracy: 0.5505 - loss: 1.1888 - val_accuracy: 0.5390 - val_loss: 1.2212
Epoch 6/60
449/449 ━━━━━━━━━━━━━━━━━━━━ 52s 115ms/step - accuracy: 0.5656 - loss: 1.1400 - val_accuracy: 0.5529 - val_loss: 1.1809
Epoch 7/60
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 112ms/step - accuracy: 0.5965 - loss: 1.0656 - val_accuracy: 0.5593 - val_loss: 1.1813
Epoch 8/60
449/449 ━━━━━━━━━━━━━━━━━━━━ 49s 108ms/step - accuracy: 0.6166 - loss: 1

In [5]:
#Testing
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

#Metrics calculation
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step
Accuracy: 0.5711897464474784
Precision: 0.5705591356168144
Recall: 0.5711897464474784
F1 Score: 0.569442013230663


In [6]:
model.save('cnn_FER2013.h5')